## Initial Setup

In [ ]:
pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install datasets

In [ ]:
# Install required packages
import subprocess
import sys

def install_packages():
    packages = [
        "bitsandbytes>=0.41.1",
        "transformers>=4.35.0",
        "peft>=0.6.0",
        "accelerate>=0.23.0",
        "datasets>=2.14.0",
        "trl>=0.7.2",
        "scipy>=1.11.3",
        "sentencepiece>=0.1.99",
        "protobuf>=4.23.4",
        "einops>=0.7.0"
    ]

    print("Installing required packages...")
    for package in packages:
        print(f"Installing {package}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    print("All packages installed successfully!")

In [ ]:
# Run package installation
install_packages()

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    TaskType
)
from trl import SFTTrainer

# Configuration
MODEL_ID = "mistralai/Mistral-7B-v0.1"
DATASET_ID = "dzunggg/legal-qa-v1"
OUTPUT_DIR = "./lora_legal_adapter"
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 2e-4
NUM_TRAIN_EPOCHS = 1
MAX_SEQ_LENGTH = 512
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

Installing required packages...
Installing bitsandbytes>=0.41.1
Installing transformers>=4.35.0
Installing peft>=0.6.0
Installing accelerate>=0.23.0
Installing datasets>=2.14.0
Installing trl>=0.7.2
Installing scipy>=1.11.3
Installing sentencepiece>=0.1.99
Installing protobuf>=4.23.4
Installing einops>=0.7.0
All packages installed successfully!


In [ ]:
pip install -U bitsandbytes

In [ ]:
!pip install -U trl

## Dataset Prep

In [ ]:
# Load the dataset
dataset = load_dataset(DATASET_ID)
print(f"Dataset loaded: {dataset}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


legal_qa_full.csv:   0%|          | 0.00/6.21M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3742 [00:00<?, ? examples/s]

Dataset loaded: DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 3742
    })
})


In [ ]:
dataset['train'][0]

{'question': 'Q: I was wondering if a pain management office is acting illegally/did an illegal action.. I was discharged as a patient from a pain management office after them telling me that a previous pain management specialist I saw administered a steroid shot wrong and I told them in the portal that I spoke to lawyers for advice but no lawsuit/case was created. It was maybe 1-2 months after I was discharged that I no longer have access to my patient portal with them. Every time I try to login I enter my credentials, wait a few seconds, and then I get re-directed back to the original screen where I have various options to login. I know I can speak to the office directly and ask them about what specifically is going on, talk to other lawyers if this is a violation of my rights, etc. but I was just wondering if anyone on this site would know if this action is in fact illegal. ',
 'answer': "A:In Kentucky, your situation raises questions about patient rights and medical records access.

In [ ]:
dataset['train'][0]['answer'][2:]

"In Kentucky, your situation raises questions about patient rights and medical records access. If you were discharged from a pain management office and subsequently lost access to your patient portal, it's important to understand your rights regarding medical records. Under the Health Insurance Portability and Accountability Act (HIPAA), you have the right to access your own medical records. A healthcare provider cannot deny you access to your records, even if you were discharged from their practice. The issue with the patient portal could be a technical glitch or a deliberate action, but either way, it needs to be addressed. You should contact the pain management office directly to inquire about the issue with the patient portal. They are obligated to provide you with a way to access your medical records, which may include providing paper copies or access through a different electronic means. If the office is unresponsive or refuses to provide access to your records, consider speaking

In [ ]:

def format_instruction(example):
    """Format the example into an instruction format suitable for fine-tuning."""

    user_input = example["question"][3:]
    assistant_response = example["answer"][2:]

    formatted_text = f"""<|im_start|>user
{user_input}<|im_end|>
<|im_start|>assistant
{assistant_response}<|im_end|>"""

    return {"text": formatted_text}

first_split = "train"

formatted_dataset = dataset[first_split].map(
    format_instruction,
    remove_columns=dataset[first_split].column_names
)

train_val_split = formatted_dataset.train_test_split(test_size=0.1)
train_dataset = train_val_split["train"]
val_dataset = train_val_split["test"]

print(f"Training examples: {len(train_dataset)}")
print(f"Validation examples: {len(val_dataset)}")

Map:   0%|          | 0/3742 [00:00<?, ? examples/s]

Training examples: 3367
Validation examples: 375


In [ ]:
train_dataset[0]

{'text': '<|im_start|>user\n Far Back Are Bank Records Checked When Filing for Bankruptcy?<|im_end|>\n<|im_start|>assistant\nen one files for bankruptcy, that person knows or should know, that the court will be looking at the debtor’s life very closely and the debtor’s bills and payment records even closer. Under normal conditions, a Chapter 7 bankruptcy trustee or a Chapter 13 court official will want to review your bank account records and your credit loans and card account records, and your tax filings, and other financial dealings. What the inquirers are looking for are hard cash and saleable assets that can be seized to pay the debt. They will also be looking for evidence of any income or assets that are by law are exempt. Just like the debtor, the court-appointed people and the creditors are bound by both federal and state laws. Having your records available and organized tells the court and its officials that you are ready to cooperate. Although it is a difficult situation, havi

## Load base mistral-7b model

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `legal_llm_finetune` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: 

In [ ]:

compute_dtype = torch.float16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto"
    )
except Exception as e:
    print(f"Error loading model with BitsAndBytes: {e}")
    print("Trying to load without quantization...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16,
        device_map="auto"
    )

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

try:
    model = prepare_model_for_kbit_training(model)
except Exception as e:
    print(f"Error preparing model for kbit training: {e}")
    print("Continuing without prepare_model_for_kbit_training...")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Define LoRA configuration

config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
import torch

def format_prompt(example):
    return f"<|im_start|>user\n{example['question'][3:]}<|im_end|>\n<|im_start|>assistant\n"

# Number of samples to test
num_samples = 5

# Put model in eval mode
model.eval()

print("\nTesting model on few samples before fine-tuning...\n")

for i in range(num_samples):
    example = dataset['train'][i]
    prompt = format_prompt(example)

    input_ids = tokenizer(prompt, return_tensors='pt', padding=True).input_ids.cuda()

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    print(f"\n--- Example {i+1} ---")
    print(f"User Input:\n{prompt}\n")
    print(f"Human Reference:\n{example['answer'][2:]}\n")
    print(f"Model Response:\n{response}\n")
    print("-" * 60)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Testing model on few samples before fine-tuning...



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Example 1 ---
User Input:
<|im_start|>user
I was wondering if a pain management office is acting illegally/did an illegal action.. I was discharged as a patient from a pain management office after them telling me that a previous pain management specialist I saw administered a steroid shot wrong and I told them in the portal that I spoke to lawyers for advice but no lawsuit/case was created. It was maybe 1-2 months after I was discharged that I no longer have access to my patient portal with them. Every time I try to login I enter my credentials, wait a few seconds, and then I get re-directed back to the original screen where I have various options to login. I know I can speak to the office directly and ask them about what specifically is going on, talk to other lawyers if this is a violation of my rights, etc. but I was just wondering if anyone on this site would know if this action is in fact illegal. <|im_end|>
<|im_start|>assistant


Human Reference:
In Kentucky, your situation

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Example 2 ---
User Input:
<|im_start|>user
My houseboat was moved while I was at work and was asked to leave the property after 16 months without any notices.. Management told me that he didn’t like those kind of people that came to visit me. I went on vacation and when I came back my electrical cord was missing and my water hose was cut in pieces. He than got of hold of me and said I have your boat and that he was putting a lien on it for back rent. I never received any notices from management. He than told me that he wanted me to leave the property and that he would hold my boat until i found another parking spot for it. I finally found a parking spot a year later and I find out that he liened my houseboat and was selling it and told me to sue him and gave me his lawyers contact. What options do I have. I now am homeless and I lost my job and seeing myself falling as time goes. Any help would be appreciated. <|im_end|>
<|im_start|>assistant


Human Reference:
If your houseboat w

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Example 3 ---
User Input:
<|im_start|>user
Wondering if I could sue a dealership if I’ve had to replace my transmission 2 times within a year. I’ve had to replace my transmission now 2 different times in 10 months of owning my 2018 Ford Expedition. My first replacement was $5,200 and then 1 1/2 months later the transmission went out and now they are working on it again <|im_end|>
<|im_start|>assistant


Human Reference:
Yes, you can sue a dealership if you have had to replace your transmission 2 times within a year. Whether you will be successful depends on the facts and your presentation. You don't mention anything about any written agreements. Allowing an attorney to evaluate, organize and draft your conciliation or (small claims) complaint (up to $15,000) would be a wise investment. A well polished complaint will not only provide the judge a solid foundation to rule in your favor, but a detailed outline on which to base your oral argument. It would provide you a distinct advant

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Example 4 ---
User Input:
<|im_start|>user
Do I need to file for CS & custody through Indian tribe since I divorced through there, not tribal member?. Ex husband is a tribal member, we had written a parental agreement including an amount he would pay monthly. It was stamped and certified, I believe, by their judge. The agreement has not been honored in over a year. I live out of the state I was divorced in (AZ), Id like to file here if its possible. Ex also lives out of the state we were divorced in. <|im_end|>
<|im_start|>assistant


Human Reference:
Once all parties have moved to another state, it is generally possible to register the order in the state where the child is now residing. Generally you need court permission to move a child out of state once a custody order has been entered (at least in Nebraska). If that did not happen before the move, then it can make matters more complicated.

Model Response:
<|im_start|>user
Do I need to file for CS & custody through Indian trib

## LoRA Fine tuning

In [ ]:
print(f"Trainable parameters: {model.print_trainable_parameters()}")

trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940
Trainable parameters: None


In [ ]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "vaibhav1/lora-mistral-legal",inference_mode=False,is_trainable=True)

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight', '

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="vaibhav1/lora-mistral-legal",
    max_steps=1000,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    optim="adamw_torch",
    save_steps=20,
    save_strategy="steps",
    logging_steps=20,
    learning_rate=LEARNING_RATE,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    eval_strategy="steps",
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    eval_steps=20,
    push_to_hub=True,
    hub_model_id="vaibhav1/lora-mistral-legal",
    hub_strategy="every_save",
    label_names=["labels"]
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=training_args,
    peft_config=config,
)


In [ ]:
 # Training the model now
print("Starting training...")

trainer.train(resume_from_checkpoint=True)

print(f"Saving model to {OUTPUT_DIR}")
trainer.save_model(OUTPUT_DIR)
print("Training completed!")

Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
20,1.829100,1.842140
40,1.786200,1.778908
60,1.814400,1.785174
80,1.715000,1.749722
100,1.794300,1.765680
120,1.703200,1.745584
140,1.754600,1.731519
160,1.774800,1.735089
180,1.701500,1.721193
200,1.730200,1.731356


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/